In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pytz
import geopandas as gpd
from datetime import datetime, timedelta
import scipy.io as sio
from tqdm import tqdm
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model
from keras.backend import clear_session
import sys
from scipy.stats import binned_statistic
from scipy.odr import ODR, Model, RealData

sys.path.append('../')  
from coastsat_keras import *

import pyfes
import os

sys.path.append('../../coastsat')  
import CoastSat_slope.SDS_slope as SDS_slope

from sklearn.linear_model import RANSACRegressor
ransac = RANSACRegressor()


transect_data = gpd.read_file('../data_coastsat/transects_edit.geojson')
transect_data = transect_data[transect_data['id'].str.contains("usa_CA")]
def linestring_to_points(feature,line):
    return line.coords

transect_data['points'] = transect_data.apply(lambda l: linestring_to_points(l['site_id'],l['geometry']), axis=1)
site_id = transect_data['id'].to_numpy()

from sklearn.model_selection import train_test_split

shore_data_new = pd.read_csv('../extended_coastsat_data/All_CA_April_2023.csvx')

sla_transect_mat = sio.loadmat('../sla/sla_transect')
sla_transect = np.squeeze(sla_transect_mat['sla_transect'])
time_sla = np.squeeze(sla_transect_mat['time_sla'])

In [ ]:
llon_1 = [np.NaN for i in range(len(transect_data['id']))] 
llon_2 = [np.NaN for i in range(len(transect_data['id']))] 
orient = [np.NaN for i in range(len(transect_data['id']))] 
sig_w = [np.NaN for i in range(len(transect_data['id']))] 
def linear_function(params, x):
    return params[0] * x + params[1]
    
monthly_means = np.empty((len(transect_data['id']),12))*np.NaN
j = 0

tid = 'usa_CA_0011-0014'
# tid = 'usa_CA_0279-0027'
clear_session()
tid_ = transect_data[transect_data['id'].str.contains(tid)]['points']
for tid__ in tid_:
    llon_1[j] = tid__[0]
    llon_2[j] = tid__[1]
oid_ = transect_data[transect_data['id'].str.contains(tid)]['orientation']
for oid__ in oid_:
    orient[j] = oid__

ind_slope = str(np.squeeze(np.argwhere(site_id==tid)))

shore_data,tide_heights,wave_data,dist_mop,spec1d_interp,spec1d_interp_lag_14d,spec1d_interp_lag_1mo,spec1d_interp_lag_3mo,spec1d_interp_lag_6mo,spec1d_interp_lag_1yr,spec1d_interp_lag_2yr,freqz = read_shore_data_new(tid,site_id,shore_data_new,time_sla,sla_transect)
df, df_time = create_df_for_keras(shore_data,tide_heights,wave_data,spec1d_interp,spec1d_interp_lag_14d,spec1d_interp_lag_1mo,spec1d_interp_lag_3mo,spec1d_interp_lag_6mo,spec1d_interp_lag_1yr,spec1d_interp_lag_2yr,freqz,tid)
df_proj,mop_time = create_proj_data(tid,site_id)
X_all,X_train,X_val,X_test,y_all,y_train,y_val,y_test,df_mean,df_std,X_proj = create_train_val_test_data(df,df_proj)

model = read_keras_model(tid,site_id)
    
pred = model.predict(X_proj)
X_tide_proj = X_proj.copy()
X_tide_proj[:,0] = -1*df_mean['tide']/df_std['tide']
pred_tide = model.predict(X_tide_proj,verbose=0)
shore_profiles = (pred - pred_tide)*df_std['shore_change'] + df_mean['shore_change']
tide_vals = X_proj[:,0]*df_std['tide'] + df_mean['tide']

idx_tide = np.logical_and(tide_vals>=df_mean['tide']-5, tide_vals<=df_mean['tide']+5)
shore_profiles = -1*np.squeeze(shore_profiles[idx_tide])
tide_vals = np.squeeze(tide_vals[idx_tide])
mop_time = np.squeeze(mop_time[idx_tide])

shore_profiles = shore_profiles - np.nanmean(shore_profiles)
tide_vals = tide_vals - np.nanmean(tide_vals)

months = mop_time.month.to_numpy()
slope_mo_ind = 2
idx_mo = np.where(months==slope_mo_ind)
shore_profiles_1 = shore_profiles[idx_mo]
tide_vals_1 = tide_vals[idx_mo]
d2 = np.nanmean(shore_profiles[idx_mo])

slope_mo_ind = 9
idx_mo = np.where(months==slope_mo_ind)
shore_profiles_2 = shore_profiles[idx_mo]
tide_vals_2 = tide_vals[idx_mo]
d1 = np.nanmean(shore_profiles[idx_mo])

shore_profiles_all = shore_profiles
tide_vals_all = tide_vals
d1 = np.nanmean(shore_profiles)

In [ ]:
in_count = 20
x_vals_ = shore_profiles_1
y_vals_ = tide_vals_1
idx_nan = ~np.isnan(x_vals_)
y_vals_ = y_vals_[idx_nan]
x_vals_ = x_vals_[idx_nan]

bin_edges = np.linspace(np.min(y_vals_), np.max(y_vals_), bin_count + 1)
y_bins, _, _ = binned_statistic(y_vals_, x_vals_, statistic='median', bins=bin_count)
y_bins_std, _, _ = binned_statistic(y_vals_, x_vals_, statistic='std', bins=bin_count)
y_bins_count, _, _ = binned_statistic(y_vals_, x_vals_, statistic='count', bins=bin_count)
x_bins = 0.5 * (bin_edges[:-1] + bin_edges[1:])
x_bins = x_bins[y_bins_count>1.]
y_bins = y_bins[y_bins_count>1.]
y_bins_std = y_bins_std[y_bins_count>1.]
y_bins_count = y_bins_count[y_bins_count>1.]
linear_model = Model(linear_function)
x_err = y_bins_std
# x_err = np.divide(y_bins_std,np.sqrt(y_bins_count))
data = RealData(y_bins, x_bins, sx=x_err*0.0 + 1., sy=x_err)
x_bins_1 = x_bins
y_bins_1 = y_bins
x_err_1 = x_err
odr = ODR(data, linear_model, beta0=[0., 1.])
    # # Run the regression.
output = odr.run()
slope_1 = output.beta[0]
intercept_1 = output.beta[0]

x_vals_ = shore_profiles_2
y_vals_ = tide_vals_2
idx_nan = ~np.isnan(x_vals_)
y_vals_ = y_vals_[idx_nan]
x_vals_ = x_vals_[idx_nan]

bin_edges = np.linspace(np.min(y_vals_), np.max(y_vals_), bin_count + 1)
y_bins, _, _ = binned_statistic(y_vals_, x_vals_, statistic='median', bins=bin_count)
y_bins_std, _, _ = binned_statistic(y_vals_, x_vals_, statistic='std', bins=bin_count)
y_bins_count, _, _ = binned_statistic(y_vals_, x_vals_, statistic='count', bins=bin_count)
x_bins = 0.5 * (bin_edges[:-1] + bin_edges[1:])
x_bins = x_bins[y_bins_count>1.]
y_bins = y_bins[y_bins_count>1.]
y_bins_std = y_bins_std[y_bins_count>1.]
y_bins_count = y_bins_count[y_bins_count>1.]
linear_model = Model(linear_function)
x_err = y_bins_std
# x_err = np.divide(y_bins_std,np.sqrt(y_bins_count))
data = RealData(y_bins, x_bins, sx=x_err*0.0 + 1., sy=x_err)
x_bins_2 = x_bins
y_bins_2 = y_bins
x_err_2 = x_err
odr = ODR(data, linear_model, beta0=[0., 1.])
    # # Run the regression.
output = odr.run()
slope_2 = output.beta[0]
intercept_2 = output.beta[0]

diff = (d1-d2)*df_std['shore_change']

x_vals_ = shore_profiles_all
y_vals_ = tide_vals_all
idx_nan = ~np.isnan(x_vals_)
y_vals_ = y_vals_[idx_nan]
x_vals_ = x_vals_[idx_nan]

bin_edges = np.linspace(np.min(y_vals_), np.max(y_vals_), bin_count + 1)
y_bins, _, _ = binned_statistic(y_vals_, x_vals_, statistic='median', bins=bin_count)
y_bins_std, _, _ = binned_statistic(y_vals_, x_vals_, statistic='std', bins=bin_count)
y_bins_count, _, _ = binned_statistic(y_vals_, x_vals_, statistic='count', bins=bin_count)
x_bins = 0.5 * (bin_edges[:-1] + bin_edges[1:])
x_bins = x_bins[y_bins_count>1.]
y_bins = y_bins[y_bins_count>1.]
y_bins_std = y_bins_std[y_bins_count>1.]
y_bins_count = y_bins_count[y_bins_count>1.]
linear_model = Model(linear_function)
x_err = y_bins_std
# x_err = np.divide(y_bins_std,np.sqrt(y_bins_count))
data = RealData(y_bins, x_bins, sx=x_err*0.0 + 1., sy=x_err)
x_bins_all = x_bins
y_bins_all = y_bins
x_err_all = x_err
odr = ODR(data, linear_model, beta0=[0., 1.])
    # # Run the regression.
output = odr.run()
slope_all = output.beta[0]
intercept_all = output.beta[0]

In [ ]:
plt.close('all')
fig, (ax0,ax1) = plt.subplots(nrows=1,ncols=2)
plt.subplots_adjust(wspace=0.15)
plt.subplots_adjust(hspace=0.15)
fig.set_size_inches(10,4)

mop_data = sio.loadmat('mop/MOP_582_profiles.mat')
x_slope = np.squeeze(mop_data['SX1D'])
z_beach = mop_data['SZ1DMeanT_msl']
z_beach_mean = np.nanmean(z_beach,axis=1)
x_min = x_slope[np.nanargmin(np.abs(z_beach_mean))]

N = 500

ax0.plot(x_slope - x_min,z_beach,color=sns.color_palette("Paired")[4],linewidth=0.2)
ax0.plot(x_slope - x_min,z_beach[:,0]+1000,color=sns.color_palette("Paired")[4],linewidth=0.8,label='All Survey Observations')
ax0.plot(x_slope - x_min,z_beach_mean,color=sns.color_palette("Paired")[5],label='Survey Average')

ax0.plot((pred - pred_tide)*df_std['shore_change'],X_proj[:,0]*df_std['tide']+df_mean['tide'],linewidth=0.1,
         alpha=0.3,color='black')
ax0.plot((pred - pred_tide)*df_std['shore_change']-1000,X_proj[:,0]*df_std['tide']+df_mean['tide'],linewidth=0.1,
         alpha=0.3,marker='.',markersize=2,linestyle='',color='black',label='All DNN Predictions')
ax0.errorbar(-1.*y_bins_all,x_bins_all,xerr=x_err_all,marker='.',linestyle='',alpha=0.3,color='black',label='Binned DNN Values')
ax0.plot(-1.*y_bins_all,y_bins_all*slope_all + intercept_all,color='black',marker='',linestyle='dashed',label='DNN Linear Fit')

x_mhhw = x_slope[np.nanargmin(np.abs(z_beach_mean-0.792))] - x_min
x_mllw = x_slope[np.nanargmin(np.abs(z_beach_mean+0.832))] - x_min

ax0.axvline(x=0,linewidth=1,color=[0.5,0.5,0.5],linestyle='dashed')
ax0.axvline(x=x_mhhw,linewidth=0.792,color=[0.5,0.5,0.5],linestyle='dashed')
ax0.axvline(x=x_mllw,linewidth=0.792,color=[0.5,0.5,0.5],linestyle='dashed')
ax0.text(x_mllw-6,-1.8,'MLLW',fontsize=12,color=[0.5,0.5,0.5],backgroundcolor='white')
ax0.text(0-4,-1.8,'MSL',fontsize=12,color=[0.5,0.5,0.5],backgroundcolor='white')
ax0.text(x_mhhw-5,-1.8,'MHHW',fontsize=12,color=[0.5,0.5,0.5],backgroundcolor='white')
ax0.text(-29,1.7,'(a)',fontsize=12,color='k')
ax0.set_ylabel('Height (m)')
ax0.set_xlabel('Distance from mean sea level (MSL)\nin m')
ax0.set_ylim([-2,2])
ax0.set_xlim([-30,58])
ax0.legend(loc='upper right', prop={'size': 10}, markerscale=2)

tide_amp = np.squeeze(np.array(CA_keras_slopes_mse['tide_amp']))

ax1.plot(-1.*shore_profiles_2,tide_vals_2,'.',color=sns.color_palette("Paired")[8],label='DNN Sep\nPredictions')
ax1.plot(-1.*shore_profiles_1,tide_vals_1,'.',color=sns.color_palette("Paired")[6],label='DNN Feb\nPredictions')

ax1.errorbar(-1.*y_bins_2,x_bins_2,xerr=x_err_2,color=sns.color_palette("Paired")[9],marker='.',linestyle='',label='Binned Sep Values')
ax1.errorbar(-1.*y_bins_1,x_bins_1,xerr=x_err_1,color=sns.color_palette("Paired")[7],marker='.',linestyle='',label='Binned Feb Values')
ax1.plot(-1.*y_bins_2,y_bins_2*slope_2 + intercept_2,color=sns.color_palette("Paired")[9],marker='',linestyle='dashed',label='Sep Linear Fit')
ax1.plot(-1.*y_bins_1,y_bins_1*slope_1 + intercept_1,color=sns.color_palette("Paired")[7],marker='',linestyle='dashed',label='Feb Linear Fit')

ax1.set_ylim([-2,2])
ax1.set_xlim([-30,58])

ax1.axvline(x=0,linewidth=1,color=[0.5,0.5,0.5],linestyle='dashed')
ax1.axvline(x=x_mhhw,linewidth=0.792,color=[0.5,0.5,0.5],linestyle='dashed')
ax1.axvline(x=x_mllw,linewidth=0.792,color=[0.5,0.5,0.5],linestyle='dashed')
ax1.text(x_mllw-6,-1.8,'MLLW',fontsize=12,color=[0.5,0.5,0.5],backgroundcolor='white')
ax1.text(0-4,-1.8,'MSL',fontsize=12,color=[0.5,0.5,0.5],backgroundcolor='white')
ax1.text(x_mhhw-5,-1.8,'MHHW',fontsize=12,color=[0.5,0.5,0.5],backgroundcolor='white')

ax1.text(-29,1.7,'(b)',fontsize=12,color='k')

# ax1.set_ylabel('Height (m)')
ax1.set_xlabel('Distance from mean sea level (MSL)\nin m')
ax1.legend(loc='upper right', prop={'size': 10}, markerscale=2)

plt.savefig('TPSB_predictions_slope.pdf',bbox_inches='tight')
plt.savefig('TPSB_predictions_slope.png',bbox_inches='tight')

plt.show()